## LangChain MCP Agent Client

### Install dependencies

In [71]:

!uv add langchain-mcp-adapters

Resolved 99 packages in 0.87ms
Audited 93 packages in 0.04ms


In [72]:
!uv add langgraph langchain-anthropic python-dotenv langchain_community langchain

Resolved 99 packages in 0.51ms
Audited 93 packages in 0.05ms


## Load env variables

In [80]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get("ANTHROPIC_API_KEY")

## Creating Server Params For stdio Connection

In [ ]:
import asyncio
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.chat_models import ChatAnthropic

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_anthropic import ChatAnthropic

### Model

In [82]:
model = ChatAnthropic(model="claude-3-5-sonnet-latest", api_key=api_key)

In [84]:
# server_params = StdioServerParameters(
#     command="python",
#     args=["C:\\Users\gaura\mcp_server101\weather\weather.py"],
# )

# async with stdio_client(server_params) as (read, write):
#     async with ClientSession(read, write) as session:
#         # Initialize the connection
#         await session.initialize()

#         # Get tools
#         tools = await load_mcp_tools(session)

#         # Create and run the agent
#         agent = create_react_agent(model, tools)
#         agent_response = await agent.ainvoke({"messages": "What is the current weather in California"})

In [85]:
from langchain_mcp_adapters.client import MultiServerMCPClient

In [86]:
async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    agent_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

In [87]:
from IPython.display import display, Markdown
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

In [88]:
agent_response

{'messages': [HumanMessage(content='what is the weather in nyc?', additional_kwargs={}, response_metadata={}, id='837f2644-fec3-471f-b15f-44704c2011be'),
  AIMessage(content=[{'text': "I'll help you get the weather forecast for New York City. I'll use the `get_forecast` function, which requires latitude and longitude coordinates for NYC. The coordinates for New York City are approximately 40.7128°N, 74.0060°W.", 'type': 'text'}, {'id': 'toolu_01Ln318gv4qnwM9N52vHFaFZ', 'input': {'latitude': 40.7128, 'longitude': -74.006}, 'name': 'get_forecast', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01HbDPsQJHxftnPBJthJJ3wi', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 507, 'output_tokens': 135}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run-a4da994b-cb7b-4d92-a26b-7cfda059eff2-0', tool_calls=[{'name': 'get_forecast', 'args': {

In [89]:
for response in agent_response["messages"]:
    user = ""
    
    if isinstance(response, HumanMessage):
        user = "**User**"
    elif isinstance(response, ToolMessage):
        user = "**Tool**"
    elif isinstance(response, AIMessage):
        user = "**AI**"
        
    if isinstance(response.content, list):
        display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
        continue
    display(Markdown(f"{user}: {response.content}"))

**User**: what is the weather in nyc?

**AI**: I'll help you get the weather forecast for New York City. I'll use the `get_forecast` function, which requires latitude and longitude coordinates for NYC. The coordinates for New York City are approximately 40.7128°N, 74.0060°W.

**Tool**: 
Today:
Temperature: 53°F
Wind: 9 to 14 mph S
Forecast: Rain. Cloudy, with a high near 53. South wind 9 to 14 mph, with gusts as high as 24 mph. Chance of precipitation is 100%. New rainfall amounts between a half and three quarters of an inch possible.

---

Tonight:
Temperature: 45°F
Wind: 8 mph SW
Forecast: A chance of rain before 8pm. Partly cloudy, with a low around 45. Southwest wind around 8 mph. Chance of precipitation is 50%. New rainfall amounts less than a tenth of an inch possible.

---

Tuesday:
Temperature: 55°F
Wind: 8 to 14 mph W
Forecast: Sunny, with a high near 55. West wind 8 to 14 mph.

---

Tuesday Night:
Temperature: 41°F
Wind: 5 to 10 mph W
Forecast: A chance of rain and snow after 8pm. Mostly cloudy, with a low around 41. West wind 5 to 10 mph. Chance of precipitation is 40%. New rainfall amounts less than a tenth of an inch possible.

---

Wednesday:
Temperature: 49°F
Wind: 7 to 12 mph NW
Forecast: A chance of rain and snow before 2pm. Partly sunny, with a high near 49. Northwest wind 7 to 12 mph. Chance of precipitation is 40%. New rainfall amounts less than a tenth of an inch possible.


**AI**: According to the forecast, today in New York City:
- Current temperature is 53°F
- It's rainy and cloudy
- There's a south wind of 9 to 14 mph with gusts up to 24 mph
- 100% chance of rain with expected rainfall between 0.5 and 0.75 inches

The rain should taper off tonight, with tomorrow (Tuesday) being sunny with a high of 55°F. However, there's a chance of mixed rain and snow starting Tuesday night into Wednesday.

In [90]:
async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    
    # Stream the response chunks
    async for chunk in agent.astream({"messages": "what is the weather in nyc?"}):
        # Extract the message content from the AddableUpdatesDict structure
        if 'agent' in chunk and 'messages' in chunk['agent']:
            for message in chunk['agent']['messages']:
                if isinstance(message, AIMessage):
                    # Handle different content formats
                    if isinstance(message.content, list):
                        # For structured content with text and tool use
                        for item in message.content:
                            if isinstance(item, dict) and 'text' in item:
                                display(Markdown(f"**AI**: {item['text']}"))
                    else:
                        # For simple text content
                        display(Markdown(f"**AI**: {message.content}"))
                        
        elif 'tools' in chunk and 'messages' in chunk['tools']:
            for message in chunk['tools']['messages']:
                if hasattr(message, 'name') and hasattr(message, 'content'):
                    # Display tool response
                    display(Markdown(f"**Tool ({message.name})**: {message.content}"))

**AI**: I can help you get the weather forecast for New York City. I'll use the get_forecast function with NYC's approximate coordinates (latitude 40.7128° N, longitude 74.0060° W).

**Tool (get_forecast)**: 
Today:
Temperature: 53°F
Wind: 9 to 14 mph S
Forecast: Rain. Cloudy, with a high near 53. South wind 9 to 14 mph, with gusts as high as 24 mph. Chance of precipitation is 100%. New rainfall amounts between a half and three quarters of an inch possible.

---

Tonight:
Temperature: 45°F
Wind: 8 mph SW
Forecast: A chance of rain before 8pm. Partly cloudy, with a low around 45. Southwest wind around 8 mph. Chance of precipitation is 50%. New rainfall amounts less than a tenth of an inch possible.

---

Tuesday:
Temperature: 55°F
Wind: 8 to 14 mph W
Forecast: Sunny, with a high near 55. West wind 8 to 14 mph.

---

Tuesday Night:
Temperature: 41°F
Wind: 5 to 10 mph W
Forecast: A chance of rain and snow after 8pm. Mostly cloudy, with a low around 41. West wind 5 to 10 mph. Chance of precipitation is 40%. New rainfall amounts less than a tenth of an inch possible.

---

Wednesday:
Temperature: 49°F
Wind: 7 to 12 mph NW
Forecast: A chance of rain and snow before 2pm. Partly sunny, with a high near 49. Northwest wind 7 to 12 mph. Chance of precipitation is 40%. New rainfall amounts less than a tenth of an inch possible.


**AI**: Currently in New York City, it's 53°F with rain and cloudy conditions. There's a 100% chance of precipitation with expected rainfall between 0.5-0.75 inches. The wind is coming from the south at 9-14 mph with gusts up to 24 mph.

Tonight, the rain should taper off by 8 PM, with partly cloudy skies and a low of 45°F. Tomorrow (Tuesday) looks better with sunny conditions and a high of 55°F, though by Tuesday night there's a chance of rain and snow.

Would you like to know about any weather alerts for New York state as well?